In [1]:
import pandas, numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
import re, os
from IPython.display import HTML, display
from tqdm import tqdm
from collections import defaultdict

from pynumdiff.utils.simulate import sine, triangle, pop_dyn, linear_autonomous, pi_cruise_control, lorenz_x
from pynumdiff.utils.evaluate import rmse, error_correlation
from pynumdiff.finite_difference import finitediff
from pynumdiff.smooth_finite_difference import mediandiff, meandiff, gaussiandiff, friedrichsdiff, butterdiff
from pynumdiff.polynomial_fit import splinediff, polydiff, savgoldiff
from pynumdiff.total_variation_regularization import tvrdiff, smooth_acceleration
from pynumdiff.linear_model import spectraldiff
from pynumdiff.kalman_smooth import rtsdiff
from pynumdiff.optimize import optimize

In [2]:
random_seeds = [1, 7, 31, 45, 57, 101, 256, 343, 737, 1024, 10000]# 11 of these; add more? -5, -25, -4, -80] # 15 of these
dts = [0.005, 0.01, 0.02, 0.04]
noise_types = [('normal', [0, 0.1]), ('laplace', [0, 0.1]), ('uniform', [-0.2, 0.2])]
noise_scales = [0.5, 1, 2, 4]
cutoff_frequencies = [1, 2, 3, 4, 5] # high frequency of signal in the data.

In [3]:
methods = [(meandiff, 'MeanDiff'),
			(mediandiff, 'MedianDiff'),
			(gaussiandiff, 'GaussianDiff'),
			(friedrichsdiff, 'FriedrichsDiff'),
			(butterdiff, 'ButterDiff'),
			(splinediff, 'SplineDiff'),
			(polydiff, 'PolyDiff'),
			(savgoldiff, 'SavGolDiff'),
			(finitediff, 'IteratedFD'), # skip first_order, because it's not going to be the best
			(tvrdiff, 'TVR'),
			#(smooth_acceleration, 'SmoothAccelTVR'), # skip in plotting?
			(rtsdiff, 'RTS'),
			(spectraldiff, 'SpectralDiff')]
sims = [(pi_cruise_control, 'Cruise Control'),
		(sine, 'Sum of Sines'),
		(triangle, 'Triangles'),
		(pop_dyn, 'Logistic Growth'),
		(linear_autonomous, 'Linear Autonomous'),
		(lorenz_x, 'Lorenz First Dimension')]

In [4]:
def main_loop(dt, cutoff_frequency, noise_type, noise_params, noise_scale, outliers, random_seed):
	"""Create and save a dataframe of results over all methods for all sims, given a certain
	hyperparameterization. Each one of these takes about 11 minutes on my machine"""
	try: # short circuit if the file already exists, just read in and display
		res = pandas.read_csv(f"~/Desktop/results/{cutoff_frequency}_{dt}_{noise_type}_{noise_scale}_{outliers}_{random_seed}.csv",
							 index_col=0)
		print(".", end='')
		
	except FileNotFoundError:
		tvgamma = np.exp(-1.6*np.log(cutoff_frequency) -0.71*np.log(dt) - 5.1)
		res = pandas.DataFrame(index=[x[1] for x in methods], columns=[x[1] for x in sims])
		for sim,sname in sims:
			x, x_truth, dxdt_truth = sim(duration=4, dt=dt, noise_type=noise_type,
										noise_parameters=noise_scale*np.array(noise_params),
										outliers=outliers, random_seed=random_seed)
			#t = np.arange(0,dt*len(x), dt)
		
			for method,mname in tqdm(methods):
				best_params, best_rmse = optimize(method, x, dt, dxdt_truth=dxdt_truth, metric='rmse')
				#best_params, best_ec = optimize(method, x, dt, dxdt_truth=dxdt_truth, metric='error_correlation')
				# best_params, best_score = optimize(method, x, dt, tvgamma=tvgamma,
				# 			search_space_updates=({'order':{2,3}} if method==tvrdiff else {})) # because TVR with order 1 hacks the cost function
				x_hat, dxdt_hat = method(x, dt, **best_params)
		
				_, _, rms_dxdt = rmse(x, dt, x_hat, dxdt_hat, dxdt_truth=dxdt_truth)
				ec = error_correlation(dxdt_hat, dxdt_truth)
				res.loc[mname, sname] = f"RMSE: {rms_dxdt:.5g}<br/>R^2: {ec:.5g}"
	
		res.to_csv(f"~/Desktop/results/{cutoff_frequency}_{dt}_{noise_type}_{noise_scale}_{outliers}_{random_seed}.csv")

		display(HTML(res.style.set_caption(
			f"cutoff_frequency={cutoff_frequency}, dt={dt}, noise_type={noise_type}, noise_scale={noise_scale}, outliers={outliers}, random_seed={random_seed}").set_properties(
			**{'text-align': 'left'}).to_html(escape=False)))
	

In [ ]:
# Main experiments here. The hypercube was getting too dang big, so take intentional slices to answer
# specific questions.

for random_seed in random_seeds:
	# Does dt matter? Vary dt, keeping everything else constant
	for dt in dts: # 4
		main_loop(dt=dt, cutoff_frequency=3, noise_type='normal', noise_params=[0, 0.1], noise_scale=1,
				  outliers=False, random_seed=random_seed)

	# Does noise type matter? Vary noise type, keeping everything else constant
	for noise_type,noise_params in noise_types: # 3
		main_loop(dt=0.01, cutoff_frequency=3, noise_type=noise_type, noise_params=noise_params, noise_scale=1,
				  outliers=False, random_seed=random_seed)
	
	# Does noise scale matter? Vary noise scale, keeping everything else constant
	for noise_scale in noise_scales: # 4
		main_loop(dt=0.01, cutoff_frequency=3, noise_type='normal', noise_params=[0,0.1], noise_scale=noise_scale,
				  outliers=False, random_seed=random_seed)
	
	# Does the presence of outliers matter? Vary presence of outliers, keeping everything else constant
	for outliers in [False, True]: # 2 Maybe try this one with more noise types too?
		main_loop(dt=0.01, cutoff_frequency=3, noise_type='normal', noise_params=[0, 0.1], noise_scale=1,
				  outliers=outliers, random_seed=random_seed)
	
	# Does the cutoff frequency matter? Vary cutoff, keeping everything else constant
	# for cutoff_frequency in cutoff_frequencies: # 5
	# 	main_loop(dt=0.01, cutoff_frequency=cutoff_frequency, noise_type='normal', noise_params=[0, 0.1], noise_scale=1,
	# 			  outliers=False, random_seed=random_seed)
	
	# Because they all pass through the central point, the number of unique runs here is 4 + 3 + 2 + 1 + 2 = 12 * 11 minutes = 2 hours
	# repeat for each random seed = 10 hours?
					

........................................................................................

100%|███████████████████████████████████████████| 12/12 [01:39<00:00,  8.29s/it]


,Cruise Control,Sum of Sines,Triangles,Logistic Growth,Linear Autonomous,Lorenz First Dimension
MeanDiff,RMSE: 1.4962R^2: 0.40062,RMSE: 1.4756R^2: 0.37433,RMSE: 1.3221R^2: 0.43799,RMSE: 0.16645R^2: 0.56396,RMSE: 2.0048R^2: 0.14552,RMSE: 2.1218R^2: 0.48995
MedianDiff,RMSE: 2.9104R^2: 0.50368,RMSE: 3.284R^2: 0.12541,RMSE: 2.3988R^2: 0.32234,RMSE: 1.6129R^2: 0.0010173,RMSE: 2.9078R^2: 0.071928,RMSE: 3.3385R^2: 0.37096
GaussianDiff,RMSE: 1.486R^2: 0.33345,RMSE: 1.4807R^2: 0.3904,RMSE: 1.2777R^2: 0.30035,RMSE: 0.1683R^2: 0.56965,RMSE: 1.9652R^2: 0.28367,RMSE: 2.0372R^2: 0.31837
FriedrichsDiff,RMSE: 1.4939R^2: 0.38179,RMSE: 1.4769R^2: 0.29753,RMSE: 1.2679R^2: 0.36282,RMSE: 0.16677R^2: 0.56644,RMSE: 1.9634R^2: 0.26396,RMSE: 2.0412R^2: 0.33647
ButterDiff,RMSE: 1.5643R^2: 0.34721,RMSE: 0.78666R^2: 0.12453,RMSE: 1.2303R^2: 0.17488,RMSE: 0.10926R^2: 0.48913,RMSE: 0.69889R^2: 0.081672,RMSE: 1.9449R^2: 0.095046
SplineDiff,RMSE: 1.848R^2: 0.32019,RMSE: 1.1377R^2: 0.037769,RMSE: 1.2627R^2: 0.13246,RMSE: 0.18979R^2: 0.3263,RMSE: 1.5741R^2: 0.16622,RMSE: 2.3338R^2: 0.071995
PolyDiff,RMSE: 1.631R^2: 0.33669,RMSE: 0.90248R^2: 0.063678,RMSE: 1.2961R^2: 0.10219,RMSE: 0.12971R^2: 0.12815,RMSE: 1.0001R^2: 0.02796,RMSE: 2.189R^2: 0.053908
SavGolDiff,RMSE: 1.4797R^2: 0.32605,RMSE: 0.92768R^2: 0.076808,RMSE: 1.2086R^2: 0.19814,RMSE: 0.14871R^2: 0.61504,RMSE: 0.88524R^2: 0.0073996,RMSE: 1.9926R^2: 0.2489
IteratedFD,RMSE: 1.4876R^2: 0.36864,RMSE: 1.219R^2: 0.38579,RMSE: 1.2569R^2: 0.24579,RMSE: 0.19214R^2: 0.21903,RMSE: 1.3196R^2: 0.18972,RMSE: 2.0679R^2: 0.33536
TVR,RMSE: 1.6333R^2: 0.44217,RMSE: 0.92549R^2: 0.093968,RMSE: 1.0516R^2: 0.16701,RMSE: 0.16522R^2: 0.27904,RMSE: 0.83255R^2: 0.0058523,RMSE: 2.0812R^2: 0.067797


.

 17%|███████▎                                    | 2/12 [00:05<00:28,  2.85s/it]Process SpawnPoolWorker-1073:
Process SpawnPoolWorker-1074:
Process SpawnPoolWorker-1076:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.13/3.13.3_1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
    ~~~~~~~~^^
  File "/usr/local/Cellar/python@3.13/3.13.3_1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/Cellar/python@3.13/3.13.3_1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/local/Cellar/python@3.13/3.13.3_1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/queues.py

In [ ]:
# plot results
def plot_perf(indie, vals, error_bars_gaps):
	"""
	:param str indie: the name of the independent variable
	:param list vals: values for the independent variable
	:param error_bars_gaps: how much space to give for the central symbol
	"""
	fig, ax = plt.subplots(2, 1, figsize=(30, 12), sharex=True, gridspec_kw={'hspace': 0})

	markers = ['^', 's', 'p', 'h', 'd', '2', '+', 'x', '*', r'$\gamma$', '.', 'o']
	facecolors = ['none' if i not in [5,6,7,10] else None for i in range(len(markers))] # 'none makes transparent'
	sizes = [90, 70, 90, 90, 80, 120, 100, 70, 120, 90, 70, 70]
	cmap = plt.get_cmap('turbo', 6) # Assign a unique color for each simulation
	colors = [cmap(i) for i in range(6)]; colors[0] = 'purple'; colors[-1] = 'red'
	colors[2] = [x*0.8 for x in colors[2]]; colors[3] = mcolors.to_rgb('gold'); colors[3] = [x*0.9 for x in colors[3]]
	colors[4] = [min(1, x*1.2) for x in colors[4]]

	point = {'cutoff_frequency':3, # all slices go through here
		'dt':0.01, 'noise_type':'normal', 'noise_scale':1, 'outliers':False}
	
	all_res = defaultdict(list)
	for random_seed in random_seeds:
		for v in vals:
			try:
				point[indie] = v # replace value
				res = pandas.read_csv(
					f"~/Desktop/results/{point['cutoff_frequency']}_{point['dt']}_{point['noise_type']}_{
						point['noise_scale']}_{point['outliers']}_{random_seed}.csv",
					index_col=0)
	
				for sim,sname in sims:
					for method,mname in methods:
						# extract the numbers
						rmse, ec = map(float, re.search(r"RMSE:\s*([\d.e+-]+)<br/>R\^2:\s*([\d.e+-]+)", res.loc[mname, sname]).groups())
						all_res[(v, mname, sname)].append((rmse, ec))

			except FileNotFoundError:
				print(f"{point['cutoff_frequency']}_{point['dt']}_{point['noise_type']}_{
						point['noise_scale']}_{point['outliers']}_{random_seed}.csv not yet computed")

	for a,v in enumerate(vals):
		for j,(sim,sname) in enumerate(sims):
			for i,(method,mname) in enumerate(methods):				
				for k,perfs in enumerate(zip(*all_res[(v, mname, sname)])):
					mean_perf = np.mean(perfs)
					std_perf = np.std(perfs)
					x = a*(len(sims)+0.3) + j + 0.075 + 0.85*i/len(methods)
					ax[k].scatter(x, mean_perf, color=colors[j], marker=markers[i], s=sizes[i],
								facecolor=facecolors[i], linewidth=1.5, label=mname if a==0 and a==0 and j==0 and k==0 else None) # label only once per method
					ax[k].vlines(x, mean_perf - std_perf, mean_perf - error_bars_gaps[k], color=colors[j], linewidth=1.5)
					ax[k].vlines(x, mean_perf + error_bars_gaps[k], mean_perf + std_perf, color=colors[j], linewidth=1.5)

	for k in range(2):
		for b in range(len(vals)-1):
			ax[k].axvline((len(sims)+0.3)*(b+1)-0.15, color="gray", linestyle="--", alpha=0.5)
		ax[k].tick_params(axis="x", length=0, labelsize=25) # length=0 hides tick lines
		ax[k].set_xlim(-0.15, len(vals)*(len(sims) + 0.3) - 0.15)
		ax[k].tick_params(axis='y', labelsize=18)
		ax[k].set_ylabel([r'RMSE($\mathbf{\hat{\dot{x}}}$, $\mathbf{\dot{x}}$)', 'Corr($\mathbf{\hat{\dot{x}}} - \mathbf{\dot{x}}$, $\mathbf{\dot{x}}$)'][k], fontsize=25)
	ax[0].set_xticklabels([])
	for label in ax[1].get_yticklabels(): label.set_fontstyle('italic')
	ax[0].set_ylim(0, 3.5)
	ax[1].set_ylim(0, 0.3)
	ax[1].set_xticks([(len(sims)+0.3)*(b+1)-(len(sims)+0.3)/2 - 0.15/2 for b in range(len(vals))]) 
	
	legend1 = ax[0].legend(ncol=2, columnspacing=0.5, handletextpad=0, loc='upper left', fontsize=15)
	ax[0].add_artist(legend1)
	for handle in legend1.legend_handles:
	    handle.set_edgecolor('dimgray')
	    if len(handle.get_facecolor()) == 1: handle.set_facecolor('dimgray') # for those that are filled
	sim_patches = [mpatches.Patch(color=colors[j], label=sname) for j,(sim,sname) in enumerate(sims)]
	legend2 = ax[0].legend(handles=sim_patches, loc='upper left', fontsize=15, bbox_to_anchor=(0.16, 1.0))

	return fig, ax


In [ ]:
fig, ax = plot_perf('dt', dts, (0.06, 0.005))
ax[1].set_xticklabels([rf"$\Delta t$={dt}" for dt in dts])
ax[1].set_ylim(0, 0.25)
ax[1].set_yticks(ax[1].get_yticks()[:-1])
fig.suptitle(r"$\Delta t$'s Effect on RMSE and Error Correlation", fontsize=32, y=0.92)
#fig.savefig("vary_dt.png", bbox_inches='tight')

In [ ]:
fig, ax = plot_perf('noise_type', [x[0] for x in noise_types], (0.05, 0.005))
ax[0].set_ylim(0, 2.5)
ax[1].set_ylim(0, 0.25)
ax[1].set_yticks(ax[1].get_yticks()[:-1])
ax[1].set_xticklabels([rf"{x[0]}" for x in noise_types])
fig.suptitle(r"Noise Type's Effect on RMSE and Error Correlaion", fontsize=32, y=0.92)
#fig.savefig("vary_noise_type.png", bbox_inches='tight')

In [ ]:
fig, ax = plot_perf('noise_scale', noise_scales, (0.05, 0.005))
ax[0].set_ylim(0, 4)
ax[1].set_ylim(0, 0.25)
ax[1].set_yticks(ax[1].get_yticks()[:-1])
ax[1].set_xticklabels([rf"scale={x}" for x in noise_scales])
fig.suptitle(r"Noise Scale's Effect on RMSE and Error Correlation", fontsize=32, y=0.92)
#fig.savefig("vary_noise_scale.png", bbox_inches='tight')

In [ ]:
fig, ax = plot_perf('outliers', [False, True], (0.09, 0.005))
ax[0].set_ylim(0, 5)
ax[1].set_ylim(0, 0.25)
ax[1].set_yticks(ax[1].get_yticks()[:-1])
ax[1].set_xticklabels(["no outliers", "with 1% of data being outliers"])
fig.suptitle(r"Outliers' Effect on RMSE and Error Correlation", fontsize=32, y=0.92)
#fig.savefig("vary_outliers.png", bbox_inches='tight')

In [ ]:
fig, ax = plot_perf('cutoff_frequency', cutoff_frequencies, (0.05, 0.005))
ax[0].set_ylim(0, 2.0)
ax[1].set_ylim(0, 0.65)
ax[1].set_xticklabels([rf"{f} Hz, $\gamma\!\approx\!${np.exp(-1.6*np.log(f) - 0.71*np.log(0.01) - 5.1):.2g}" for f in cutoff_frequencies])
fig.suptitle(r"Cutoff Frequency's Effect on RMSE and Error Correlation", fontsize=32, y=0.92)
#fig.savefig("vary_f.png", bbox_inches='tight')